In [ ]:
import transformers
import torch
import datasets
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from tqdm import tqdm

# Load CLIP

In [ ]:
# from transformers import CLIPProcessor, CLIPModel
# checkpoint = "openai/clip-vit-large-patch14"
# processor = CLIPProcessor.from_pretrained(checkpoint)
# CLIP_not_FT = CLIPModel.from_pretrained(checkpoint)

# # check if GPU is available
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print(device)
# # move model to device
# CLIP_not_FT = CLIP_not_FT.to(device)


In [ ]:
# samples = dataset[:8]
# images = samples["image"]
# texts = samples["title"]
# inputs = processor(text=texts, images=images, return_tensors="pt", padding=True)
# inputs = {name: tensor.to(device) for name, tensor in inputs.items()}
# outputs = CLIP_not_FT(**inputs, return_loss=True)
# print(outputs)

In [ ]:
# from experiment_1 import run_experiment_1

In [ ]:
# accuracy = run_experiment_1(dataset=dataset, model=CLIP_not_FT, processor=processor, device=device)
# print(accuracy)

# Fine Tune CLIP

In [1]:
import transformers
import torch
import datasets
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from tqdm import tqdm

In [2]:
# load dataset from huggingface
dataset = datasets.load_dataset("cringgaard/boatsV2", use_auth_token=True)
dataset = dataset["train"]
# rename title to text
dataset = dataset.rename_column("title", "text")
#split dataset into train and test
dataset = dataset.train_test_split(test_size=0.2)
train_dataset = dataset["train"]
test_dataset = dataset["test"]

Using custom data configuration cringgaard--boatsV2-2a63089b8a44e701
Reusing dataset parquet (C:\Users\chris\.cache\huggingface\datasets\cringgaard___parquet\cringgaard--boatsV2-2a63089b8a44e701\0.0.0\2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/1 [00:00<?, ?it/s]

In [3]:
train_dataset

Dataset({
    features: ['image', 'text', 'manufacturer', 'length', 'size category', 'year built'],
    num_rows: 10268
})

In [4]:
test_dataset

Dataset({
    features: ['image', 'text', 'manufacturer', 'length', 'size category', 'year built'],
    num_rows: 2567
})

In [5]:
# load model
from transformers import CLIPProcessor, CLIPModel
checkpoint = "openai/clip-vit-large-patch14"
processor = CLIPProcessor.from_pretrained(checkpoint)
model = CLIPModel.from_pretrained(checkpoint)

# check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
# move model to device
model = model.to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

cuda


In [6]:
# make pytorch dataset
from torch.utils.data import DataLoader

def collate_fn(examples):
    texts = [example["text"] for example in examples]
    images = [example["image"] for example in examples]
    inputs = processor(text=texts, images=images, return_tensors="pt", padding=True)
    inputs = {name: tensor.to(device) for name, tensor in inputs.items()}
    return inputs

train_dataloader = DataLoader(train_dataset, collate_fn=collate_fn, batch_size=4, shuffle=True)

# train model
from tqdm import tqdm
step = 0
for epoch in range(3):
    model.train()
    for batch in tqdm(train_dataloader):
        optimizer.zero_grad()
        outputs = model(**batch, return_loss=True)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        step += 1
        if step % 100 == 0:
            print("loss:", loss.item())
            model.save_pretrained(f"clip_finetuned{step}")            

  4%|▍         | 99/2567 [08:13<2:59:32,  4.36s/it]

loss: 1.3831353187561035


  8%|▊         | 199/2567 [15:43<2:57:15,  4.49s/it]

loss: 1.368812084197998


 12%|█▏        | 299/2567 [23:08<2:47:48,  4.44s/it]

...

loss: 1.3818988800048828


 17%|█▋        | 432/2567 [30:44<2:55:52,  4.94s/it]

loss: 1.3182692527770996


 17%|█▋        | 435/2567 [31:01<2:32:03,  4.28s/it]


KeyboardInterrupt: 